# FE Weekly


## This Week Topics

**[一、设计模式](#一、设计模式)**

**[二、事件模型](#二、事件模型)**

　　**[１． 事件接口 `EventTarge`](### １． 事件接口 `EventTarge`)**

　　**[２． 事件的传播与代理机制](### ２． 事件的传播与代理机制)**

　　　　**[１）事件的传播机制](#### １）事件的传播机制)**
    
　　　　**[２）事件的代理机制](#### ２）事件的代理机制)**
    
　　**[３．事件对象 `Event`](### ３．事件对象 `Event`)**

　　　　**[１）`Event` 对象实例的属性](#### １）`Event` 对象实例的属性)**
    
　　　　**[2）`Event` 对象实例的方法](#### 2）`Event` 对象实例的方法)**

　　**[４．自定义事件对象 `CustomEvent`](### ４．自定义事件对象 `CustomEvent`)**
  
## 一、设计模式

　　什么是设计模式？我的理解是定义信息于时间和空间上如何流转的模型。今天我们要介绍的是 “观察者模式” 和 “发布订阅模式”。  
  
　　这两种模式常用于 事件响应 或 消息响应 中。比如 JS 中的事件模型所使用的就是 “发布订阅模式”。  
  
　　所谓 “观察者模式”，包含两个对象，一个是 “观察者”， 还有一个是 “发布者”。“发布者” 通常包含一个 “观察者列表”，用来存放注册上来的 “观察者”；而 “观察者” 通常有一个 `update` 方法，当有消息产生或事件触发时，发布者通过调用 “观察者列表” 中所有观察者的这个方法，并传入相关的信息作为参数（通常是事件或消息本身），来告知所有观察者，有大事要发生了啊！  
  
　　而 “发布订阅模式” 则略有不同，它也包含两个对象，一个是 “订阅者”（其实叫 “观察者” 也可以，“监听者” 也可以），另一个是 “发布者”（也可以叫 “调度者”）。“订阅者/监听者” 向 “发布者/调度者” 订阅/监听 某个 主题的事件或消息，并带上一个回调函数，后者在有消息产生或事件触发时，调用这个回调函数，并传入相关信息作为参数（如事件或消息本身），从而告知所有订阅了该主题的对象。  
  
　　可以说，“观察者模式” 注册的是观察者本身，而 “发布订阅模式” 注册（监听）的是某个主题。很显然，后者的耦合度要低一些。  
  
　　这里解释一下为什么这么说。就这两个设计模式，对于 “发布者” 而言，所注册的必然都是其所依赖的（如果不是所依赖的，则无需注册），所以可以这么说：“观察者模式” 依赖于 “观察者” 本身，而 “发布订阅模式” 则不（通过主题，或事件类型这一层，它 “解耦” 了 “发布者” 与 “订阅者”）。


## 二、事件模型

　　`Javascript` 事件模型主要由 事件接口（`EventTarget` 和 `CustomEvent`）、事件机制（传播与代理）和事件对象（`Event`对象）组成。
  
### １． 事件接口 `EventTarge`

　　“所有 节点对象 都定义有该接口” —— 这是在说：所有的节点对象都拥有 `EventTarge` 所定义的方法，比如 `addEventListener`、`removeEventListener` 和 `dispatchEvent`。

　　首先说一下 `addEventListener`。它就类似于作业当中发布订阅模式的 `subscribe` 方法。`addEventListener` 有三个参数，第一个是 `type`， 可以理解为主题，实际上是事件类型；第二个参数 `listener` 是监听函数，或者说是回调函数，也就是发布者在事件被触发时，要调用的方法；第三个参数是 `useCapture`，表示监听函数是否在捕获阶段（capture）调用，所谓捕获，在接下来的 2.事件机制 中可以我们会知道它的具体定义。

　　关于后面两个参数，还有一点要注意的是，第二个参数 `listener` 除了是监听函数，还可以是一个具有 `handleEvent` 方法的对象；第三个参数 `useCapture` 除了布尔值 `useCapture`，还可以是一个属性配置对象。具体阅读这篇[参考文档](http://javascript.ruanyifeng.com/dom/event.html#toc0)，我这边就不照本宣科了。在这篇文档中，大家也可以看到关于一个事件多个监听函数的操作，以及监听函数中，`this` 的指向问题（简单来说，`this` 指向被触发的节点对象）。
  
　　回到事件接口 `EventTarge` 本身，它还有两个方法，`removeEventListener` 和 `dispatchEvent`。前者用来移除监听者，与 `addEventListener` 是反向操作，参数必须与在监听的时候完全一致，才能正确移除，否则并未移除；后者用来触发一个事件，它的参数是一个 `event` 对象，在 3.事件对象 中会了解到它的方法和属性。

　　需要注意的是，事件的监听并不只有 `EventTarge.addEventListener` 一种方式，还可以有：

- HTML 内嵌方式。如：`<div onClick='doSomething(event)'></div>`  
（注意：`doSomething` 只会在冒泡阶段触发。缺点：不符合 html 代码 与 javascript 代码分离的原则）


- 设置元素节点的事件属性。如 `div.onClick=function (event) { doSomthing(event) };`  
（缺点：只能绑定一个函数，后浪推前浪）

　　鉴于以上两种方法都有其不完善之处，实际应用中还是推荐使用 `EventTarge.addEventListener` 的方式来进行事件监听。


### ２． 事件的传播与代理机制

#### １）事件的传播机制

　　事件的传播机制按触发顺序分为三个阶段：捕获 → 目标 → 冒泡。这三个阶段会使得同一个事件会在多个节点上触发。

　　具体来说就是：

- “捕获阶段” 就是 事件从 `window` 传到 目标节点 的过程，沿途上触发所有监听这个事件的途经节点，也可以说是被途经节点都捕获一次；

- “目标阶段” 就是 事件在 目标节点 被捕获；

- “冒泡阶段” 就是 事件从 目标节点 又传回到 `window` 节点。

　　大家结合一下[阮一峰的举例](http://javascript.ruanyifeng.com/dom/event.html#toc10)理解就很清楚了。


#### ２）事件的代理机制

　　事件的代理，也叫事件的委托，这两个名字指代的都是同一个事物。简单来说，就是：在监听一个父节点的事件，就等效于监听父节点的所有子节点的该事件，而且以后再添加子节点，监听函数依然有效。一旦理解了这点，就没什么神秘的了，它的应用场景也很显然，不用写一大堆监听代码，只要监听它们的父节点就完事了，也可以节省内存，毕竟一个监听函数就是一块内存啊。具体示例见[阮一峰的教程](http://javascript.ruanyifeng.com/dom/event.html#toc11)。


### ３．事件对象 `Event`

　　事件发生以后，会产生一个事件对象，作为参数传给监听函数。
  
　　浏览器原生提供一个 `Event` 对象，所有的事件都是这个对象的实例，或者说继承了 `Event.prototype` 对象。

　　`Event` 对象本身就是一个构造函数，可以用来生成新的实例，如：
```javascript
event = new Event(type, options);
```

　　其中，`type` 是字符串，表示事件的名称；`options` 是一个对象，表示事件对象的配置。该对象主要有下面两个属性：

- `bubbles`：布尔值，可选，默认为false，表示事件对象是否冒泡，即生成的事件是否能在“冒泡阶段”触发监听函数。

- `cancelable`：布尔值，可选，默认为false，表示事件是否可以被取消，即在捕获或冒泡过程中，能否用 `Event.preventDefault()` 取消这个事件。  
（注：一旦事件被取消，就好像从来没有发生过，不会触发浏览器对该事件的默认行为。）

　　另外要注意一下：处于 “目标阶段” 的节点不存在 “是否能在冒泡阶段触发” 的问题。这一点，在 [阮一峰老师的教程](http://javascript.ruanyifeng.com/dom/event.html#toc12) 里有示例解读。

　　`Event` 对象实例的属性和方法在这里做一个简要的总结：（具体示例可查看[阮一峰的精彩示例](http://javascript.ruanyifeng.com/dom/event.html#toc13)）

#### １）`Event` 对象实例的属性

- `Event.bubbles`：只读，布尔值，表示该事件是否会冒泡（前面说过，除非显式声明，`Event` 构造函数生成的事件，默认是不冒泡的）；

- `Event.eventPhase`：只读，整数，表示该事件目前所处的阶段（0-未发生；1-捕获；2-目标；3-冒泡）；

- `Event.cancelable`：只读，布尔值，表示该事件是否可以被取消（除非显式声明，`Event` 构造函数生成的事件，默认是不可以取消的）；

>　　注：
>
>　　当 `Event.cancelable` 属性为 `true` 时，调用 `Event.preventDefault()` 则可取消这个事件，阻止浏览器对该事件的默认行为。
>
>　　什么元素有默认行为呢？如链接 `<a>` 的默认跳转行为，提交按钮 `<input type=”submit”>` 的默认提交表单行为等。
>
>　　如果事件不能取消，调用 `Event.preventDefault()` 会没有任何效果。所以使用这个方法之前，最好用 `Event.cancelable` 属性判断一下是否可以取消。

- `Event.cancelBubble`：可写，布尔值，如果设为 `true`，相当于执行 `Event.stopPropagation()`，可以阻止事件的传播；

- `event.defaultPrevented`：只读，布尔值，表示该事件是否调用过 `Event.preventDefault` 方法；
___
- `Event.currentTarget`：事件当前所在的节点，即正在执行的监听函数所绑定的那个节点；

- `Event.target`：原始触发事件的那个节点，即事件最初发生的节点；
___
- `Event.type`：事件类型；

- `Event.timeStamp`：事件发生的时间的毫秒时间戳；

- `Event.isTrusted`：事件来自用户行为还是脚本行为；

- `Event.detail`：只有浏览器的 UI（用户界面）事件才具有。该属性返回一个数值，表示事件的某种信息。具体含义与事件类型相关。

>　　比如，对于 `click` 和 `dbclick` 事件，`Event.detail` 是鼠标按下的次数（1表示单击，2表示双击，3表示三击）；
>
>　　对于鼠标滚轮事件，`Event.detail` 是滚轮正向滚动的距离，负值就是负向滚动的距离，返回值总是 3 的倍数。

#### 2）`Event` 对象实例的方法

- `Event.preventDefault()`：阻止浏览器对该事件的默认行为，前已述。例子有：[阻止不符合要求的内容被输入到表单](http://javascript.ruanyifeng.com/dom/event.html#toc22)；

- `Event.stopPropagation()`：阻止事件在直系节点上的传播；

- `Event.stopImmediatePropagation()`：连旁系节点的传播也阻止了；

- `Event.composedPath()`：返回一个数组，数组成员是 事件的最底层节点 和 依次冒泡经过的所有上层节点。


### ４．自定义事件对象 `CustomEvent`

　　它是为了弥补浏览器预定义事件不能绑定自定义数据的缺憾而设计的。
  
　　`CustomEvent` 的配置对象除了接受 `Event` 事件的配置属性，还有有一个自己的属性 `detail` 表示事件的附带数据，默认为null。
  
`*END*`
